In [1]:
import numpy as np
import pandas as pd
import re
import shutil
import os
import unicodedata
import urllib3
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
num_samples = 33000

In [4]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [5]:
def preprocess_sentence(sent):
    sent = unicode_to_ascii(sent.lower())
    sent = re.sub(r"([?.!,¿])",r" \1", sent)
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)
    sent = re.sub(r"\s+"," ",sent)
    return sent

In [6]:
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"
print(preprocess_sentence(en_sent))
print(preprocess_sentence(fr_sent))

have you had dinner ?
avez vous deja dine ?


In [7]:
def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []
    
    with open('data/fra.txt', "r") as lines:
        for i, line in enumerate(lines):
            # source, target 분리
            src_line, tar_line, _ = line.strip().split('\t')
            
            # source 데이터 전처리
            src_line_input = [w for w in preprocess_sentence(src_line).split()]
            
            # target 데이터 전처리
            tar_line = preprocess_sentence(tar_line)
            tar_line_input = [w for w in ("<sos> "+ tar_line).split()]
            tar_line_target = [w for w in (tar_line + " <eos>").split()]
            
            encoder_input.append(src_line_input)
            decoder_input.append(tar_line_input)
            decoder_target.append(tar_line_target)
            
            if i == num_samples -1:
                break
                
    return encoder_input, decoder_input, decoder_target

In [8]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()
print(sents_en_in[:5])
print(sents_fra_in[:5])
print(sents_fra_out[:5])

[['go', '.'], ['hi', '.'], ['hi', '.'], ['run', '!'], ['run', '!']]
[['<sos>', 'va', '!'], ['<sos>', 'salut', '!'], ['<sos>', 'salut', '.'], ['<sos>', 'cours', '!'], ['<sos>', 'courez', '!']]
[['va', '!', '<eos>'], ['salut', '!', '<eos>'], ['salut', '.', '<eos>'], ['cours', '!', '<eos>'], ['courez', '!', '<eos>']]


In [9]:
tokenizer_en = Tokenizer(filters = "", lower=False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)

tokenizer_fra = Tokenizer(filters="", lower = False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)
decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)

In [10]:
encoder_input = pad_sequences(encoder_input, padding="post")
decoder_input = pad_sequences(decoder_input, padding="post")
decoder_target = pad_sequences(decoder_target, padding="post")

In [11]:
print(encoder_input.shape)
print(decoder_input.shape)
print(decoder_target.shape)

(33000, 8)
(33000, 16)
(33000, 16)


In [13]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print(f"영어 단어 집합의 크기 : {src_vocab_size}, 프랑스어 단어 집합의 크기 : {tar_vocab_size}")

영어 단어 집합의 크기 : 4663, 프랑스어 단어 집합의 크기 : 8038


In [14]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word

tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

In [15]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[11475  9081 24948 ... 24342  3627  8690]


In [16]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [17]:
n_of_val = int(num_samples *0.1)

In [20]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [21]:
print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(29700, 8)
(29700, 16)
(29700, 16)
(3300, 8)
(3300, 16)
(3300, 16)


In [23]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [24]:
latent_dim = 50

In [26]:
# encoder
encoder_inputs = Input(shape=(None, ))
enc_emb = Embedding(src_vocab_size, latent_dim)(encoder_inputs) # embedding
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

In [27]:
#decoder
decoder_inputs = Input(shape=(None, ))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim)
dec_emb = dec_emb_layer(decoder_inputs)
dec_masking = Masking(mask_value = 0.0)(dec_emb)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state = encoder_states)

decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [28]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [29]:
# 원-핫 인코딩을 하지 않은 상태로, 정수 레이블에 대해서 
# 다중 클래스 분류 문제를 풀고자 하는 경우에는 
# categorical_crossentropy함수가 아니라 sparse_categorical_crossentropy를 사용하면 됩니다. 
# 이는 케라스에서 규정한 약속입니다.

model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     233150      input_2[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     401900      input_3[0][0]                    
_______________________________________________________________________________________

In [30]:
model.fit(x = [encoder_input_train, decoder_input_train], 
          y = decoder_target_train,
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size=128,
          epochs=50)

Epoch 1/50
233/233 [==============================] - 103s 441ms/step - loss: 3.1568 - acc: 0.6061 - val_loss: 1.9228 - val_acc: 0.6432
Epoch 2/50
233/233 [==============================] - 101s 434ms/step - loss: 1.7432 - acc: 0.7241 - val_loss: 1.6267 - val_acc: 0.7407
Epoch 3/50
233/233 [==============================] - 101s 432ms/step - loss: 1.5502 - acc: 0.7486 - val_loss: 1.5250 - val_acc: 0.7564
Epoch 4/50
233/233 [==============================] - 100s 431ms/step - loss: 1.4395 - acc: 0.7639 - val_loss: 1.4237 - val_acc: 0.7694
Epoch 5/50
233/233 [==============================] - 101s 432ms/step - loss: 1.3532 - acc: 0.7792 - val_loss: 1.3483 - val_acc: 0.7848
Epoch 6/50
233/233 [==============================] - 101s 432ms/step - loss: 1.2909 - acc: 0.7889 - val_loss: 1.3113 - val_acc: 0.7896
Epoch 7/50
233/233 [==============================] - 99s 423ms/step - loss: 1.2408 - acc: 0.7955 - val_loss: 1.2686 - val_acc: 0.7959
Epoch 8/50
233/233 [=============================

In [31]:
encoder_model = Model(encoder_inputs, encoder_states)

In [32]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2= decoder_dense(decoder_outputs2)

In [33]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [34]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    
    target_seq = np.zeros((1,1))
    target_seq[0,0] = tar_to_index['<sos>']
    
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition_condition_condition_condition_condition_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] +  states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        
        decoded_sentence += ' ' + sampled_char
        
        if (sampled_char == '<eos>' or len(decoded_sentence) > 50):
            stop_condition= True
            
        target_seq = np.zeros((1,1))
        target_seq[0,0] = sampled_token_index
        
        states_value = [h,c]
        
    return decoded_sentence

In [35]:
# 원문의 정수 시ㅜ컨스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp =''
    for i in input_seq:
        if(i != 0):
            temp = temp + index_to_src[i] + ' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp = ''
    for i in input_seq:
        if( i != 0 and i != tar_to_index['<sos>'] and i != tar_to_index['<eos>']):
            temp = temp + index_to_tar[i] + ' '

    return temp

In [36]:
for seq_index in [3,50, 100, 300, 1001]:
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequencece(input_seq)

    print("원문 : ",seq2src(encoder_input_train[seq_index]))
    print("번역문 :",seq2tar(decoder_input_train[seq_index]))
    print("예측문 :",decoded_sentence[:-5])
    print("\n")

원문 :  i did a good job . 
번역문 : j ai fait du bon travail . 
예측문 :  j ai fait un bon boulot . 


원문 :  be nice . 
번역문 : soyez gentilles ! 
예측문 :  soyez bien ! 


원문 :  i like turtles . 
번역문 : j aime les tortues . 
예측문 :  j aime les tout . 


원문 :  do you eat a lot ? 
번역문 : mangez vous beaucoup ? 
예측문 :  as tu du cafe ? 


원문 :  once again . 
번역문 : encore une fois . 
예측문 :  fais une fois . 




In [37]:
for seq_index in [3,50,100,300,1001]:
    input_seq = encoder_input_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)

    print("원문 : ",seq2src(encoder_input_test[seq_index]))
    print("번역문 :",seq2tar(decoder_input_test[seq_index]))
    print("예측문 :",decoded_sentence[:-5])
    print("\n")

원문 :  why is snow white ? 
번역문 : pourquoi la neige est elle blanche ? 
예측문 :  est ce que est il ete occupe ? 


원문 :  drive on . 
번역문 : avancez ! 
예측문 :  d accord qui vous . 


원문 :  do you have the key ? 
번역문 : as tu la cle ? 
예측문 :  avez vous l air assez ? 


원문 :  they re wrong . 
번역문 : ils ont tort ! 
예측문 :  ils sont des qui . 


원문 :  i got a bargain . 
번역문 : j ai fait une affaire . 
예측문 :  j ai une voiture . 


